# Predict
Este notebook tiene de objetivo implementar las funcionalidades para así pasar la fase de predicciones

In [12]:
import tensorflow as tf
import numpy as np
import os


from grav_lens.models.load_models import load_minmaxscaler, load_ipca_low
from grav_lens.preprocess.gmm import reconstruct_batch_images

minmaxscaler = load_minmaxscaler() # usado para reescalar datos
print(minmaxscaler)
# Cargar el modelo IPCA low
ipca_low = load_ipca_low()
print(ipca_low)

# de predicción
from grav_lens.utils import load_testing_dataset, save_predictions





IncrementalPCA(batch_size=64, n_components=64)


In [17]:
# Directorio base para los datos y resultados
home_pred_data = os.path.join("..", "data", "test_data")
home_data = os.path.join("..", "data")

# Paso 2: Cargar el d
# ataset de testing
batch_size = 64  # Ajustar el tamaño del batch según sea necesario
max_files = -1  # Cargar todos los archivos de testing
test_dataset = load_testing_dataset(home_pred_data, batch_size=batch_size, max_files=max_files)


In [18]:

loaded_model = tf.keras.models.load_model(
    'models/model_1002.keras',
    safe_mode=False
)


In [19]:
import os
import pickle

checkpoint_path = "checkpoint.pkl"

# Función para guardar el checkpoint
def save_checkpoint(index):
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(index, f)

# Función para cargar el checkpoint
def load_checkpoint():
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as f:
            return pickle.load(f)
    return 0  # Si no hay checkpoint, empezar desde el inicio

# Cargar el checkpoint

#start_index = load_checkpoint()
start_index=0

In [20]:

saved_files = set()
# Paso 3: Iterar sobre el dataset para hacer predicciones desde el checkpoint
for batch_index, (X_batch, names_batch) in enumerate(test_dataset):
    print(X_batch.shape)
    # Omitir batches procesados previamente
    if batch_index < start_index:
        print(f"Omitiendo Batch {batch_index + 1}")
        continue
    
    try:
        # duplicados
        for file_name in names_batch.numpy():
            if file_name in saved_files:
                print(f"Advertencia: Archivo duplicado detectado {file_name}")
            saved_files.add(file_name)
        
        # Realizar la predicción para el batch actual
        predictions = loaded_model.predict(X_batch)
        actual_batch_size = X_batch.shape[0]  # Usar el tamaño actual del batch


        img_predictions = reconstruct_batch_images(*predictions, actual_batch_size, ipca_low)
        img_predictions = minmaxscaler.inverse_transform(img_predictions)
        
        # Paso 4: Guardar las predicciones con sus nombres correspondientes
        save_predictions(img_predictions, names_batch, home=home_data)
        
        # Guardar checkpoint después de procesar exitosamente un batch
        save_checkpoint(batch_index + 1)
        print(f"Batch {batch_index + 1} guardado exitosamente")

    except Exception as e:
        print(f"Error en el batch {batch_index + 1}: {e}")
        break  # Detener el proceso y reiniciar desde el último batch exitoso



(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step
Batch 1 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
Batch 2 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
Batch 3 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Batch 4 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
Batch 5 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
Batch 6 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Batch 7 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
Batch 8 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
Batch 9 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
Batch 10 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
Batch 11 guardado exitosamente
(64, 128, 128, 3)
2/2 ━━━━━━━━━━━━━━━━━━━